<div align="center">
<img style="display: block; margin: auto;" alt="photo" src="https://cdn.quantconnect.com/web/i/icon.png">

Quantconnect

Introduction to Financial Python
</div>

# 05 Pandas-Resampling and DataFrame

# Introduction
In the last chapter we had a glimpse of Pandas. In this chapter we will learn about resampling methods and the DataFrame object, which is a powerful tool for financial data analysis.

# Fetching Data
Here we use the QuantBook API to retrieve data.



In [2]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 7.7 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=db70554f3cf9f6ce5f66d80c9582bc3cb3a348846001c68b38ce7b0f026248de
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import yfinance as yf

aapl = yf.Ticker("AAPL")

# get stock info
print(aapl.info)

# get historical market data
aapl_table = aapl.history(start="2016-01-01",  end="2017-12-31")
aapl_table

{'zip': '95014', 'sector': 'Technology', 'fullTimeEmployees': 147000, 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand radi

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04,23.655170,24.291446,23.514544,24.286835,270597600,0.0,0
2016-01-05,24.379045,24.402098,23.609059,23.678219,223164000,0.0,0
2016-01-06,23.182563,23.599832,23.023496,23.214838,273829600,0.0,0
2016-01-07,22.749168,23.083443,22.230465,22.235075,324377600,0.0,0
2016-01-08,22.719197,22.848296,22.306540,22.352646,283192000,0.0,0
...,...,...,...,...,...,...,...
2017-12-22,41.827752,42.004949,41.784653,41.906773,65397600,0.0,0
2017-12-26,40.898671,41.059104,40.630480,40.843597,132742000,0.0,0
2017-12-27,40.731061,40.893888,40.637675,40.850788,85992800,0.0,0


In [7]:

aapl = aapl_table['Close']['2017']

In [8]:
print(aapl)

Date
2017-01-03    27.372360
2017-01-04    27.341726
2017-01-05    27.480770
2017-01-06    27.787130
2017-01-09    28.041647
                ...    
2017-12-22    41.906773
2017-12-26    40.843597
2017-12-27    40.850788
2017-12-28    40.965721
2017-12-29    40.522736
Name: Close, Length: 251, dtype: float64


Recall that we can fetch a specific data point using series['yyyy-mm-dd']. We can also fetch the data in a specific month using series['yyyy-mm'].

In [9]:
print(aapl['2017-3'])

Date
2017-03-01    33.086285
2017-03-02    32.889843
2017-03-03    33.083916
2017-03-06    32.979778
2017-03-07    33.022385
2017-03-08    32.899300
2017-03-09    32.823559
2017-03-10    32.932442
2017-03-13    32.946648
2017-03-14    32.896938
2017-03-15    33.244869
2017-03-16    33.299297
2017-03-17    33.133617
2017-03-20    33.481556
2017-03-21    33.098125
2017-03-22    33.472088
2017-03-23    33.353737
2017-03-24    33.287468
2017-03-27    33.344276
2017-03-28    34.035393
2017-03-29    34.111130
2017-03-30    34.066170
2017-03-31    34.002262
Name: Close, dtype: float64


In [10]:
aapl['2017-2':'2017-4']

Date
2017-02-01    30.341726
2017-02-02    30.289883
2017-02-03    30.419495
2017-02-06    30.704647
2017-02-07    30.996870
                ...    
2017-04-24    33.997520
2017-04-25    34.208176
2017-04-26    34.006985
2017-04-27    34.033024
2017-04-28    33.999897
Name: Close, Length: 61, dtype: float64

.head(N) and .tail(N) are methods for quickly accessing the first or last N elements.

In [11]:
print(aapl.head(5))
print(aapl.tail(10))

Date
2017-01-03    27.372360
2017-01-04    27.341726
2017-01-05    27.480770
2017-01-06    27.787130
2017-01-09    28.041647
Name: Close, dtype: float64
Date
2017-12-15    41.657734
2017-12-18    42.244404
2017-12-19    41.794231
2017-12-20    41.748737
2017-12-21    41.906773
2017-12-22    41.906773
2017-12-26    40.843597
2017-12-27    40.850788
2017-12-28    40.965721
2017-12-29    40.522736
Name: Close, dtype: float64


# Resampling
**_series.resample(freq)_** is a class called "DatetimeIndexResampler" which groups data in a Series object into regular time intervals. The argument "freq" determines the length of each interval.

**_series.resample.mean()_** is a complete statement that groups data into intervals, and then compute the mean of each interval. For example, if we want to aggregate the daily data into monthly data by mean:

In [12]:
by_month = aapl.resample('M').mean()
print(by_month)

Date
2017-01-31    28.178333
2017-02-28    31.606276
2017-03-31    33.282221
2017-04-30    33.819263
2017-05-31    36.125686
2017-06-30    35.133983
2017-07-31    35.245242
2017-08-31    37.897227
2017-09-30    37.604738
2017-10-31    37.654552
2017-11-30    41.233915
2017-12-31    41.160039
Freq: M, Name: Close, dtype: float64


We can also aggregate the data by week:

In [13]:
by_week = aapl.resample('W').mean()
print(by_week.head())

Date
2017-01-08    27.495497
2017-01-15    28.097734
2017-01-22    28.266119
2017-01-29    28.553981
2017-02-05    29.662543
Freq: W-SUN, Name: Close, dtype: float64


We can also aggregate the data by month with max:

In [14]:
aapl.resample('M').max()

Date
2017-01-31    28.739214
2017-02-28    32.451969
2017-03-31    34.111130
2017-04-30    34.264980
2017-05-31    37.099129
2017-06-30    36.944653
2017-07-31    36.471699
2017-08-31    39.129723
2017-09-30    39.141651
2017-10-31    40.332249
2017-11-30    42.050137
2017-12-31    42.244404
Freq: M, Name: Close, dtype: float64

We can choose almost any frequency by using the format 'nf', where 'n' is an integer and 'f' is M for month, W for week and D for day.

In [15]:
three_day = aapl.resample('3D').mean()
two_week = aapl.resample('2W').mean()
two_month = aapl.resample('2M').mean()


print(three_day)
print(two_week)
print(two_month )

Date
2017-01-03    27.398286
2017-01-06    27.787130
2017-01-09    28.110775
2017-01-12    28.078174
2017-01-15    28.279669
                ...    
2017-12-17    42.019318
2017-12-20    41.854094
2017-12-23          NaN
2017-12-26    40.886702
2017-12-29    40.522736
Freq: 3D, Name: Close, Length: 121, dtype: float64
Date
2017-01-08    27.495497
2017-01-22    28.172572
2017-02-05    29.108262
2017-02-19    31.516819
2017-03-05    32.595556
2017-03-19    33.017883
2017-04-02    33.625220
2017-04-16    33.849728
2017-04-30    33.791845
2017-05-14    35.697633
2017-05-28    36.496180
2017-06-11    36.515006
2017-06-25    34.529048
2017-07-09    34.265135
2017-07-23    35.278870
2017-08-06    36.272775
2017-08-20    37.904118
2017-09-03    38.458792
2017-09-17    38.282710
2017-10-01    36.840872
2017-10-15    37.071597
2017-10-29    37.757320
2017-11-12    41.032237
2017-11-26    41.204377
2017-12-10    40.867068
2017-12-24    41.620867
2018-01-07    40.795711
Freq: 2W-SUN, Name: Close, 

Besides the mean() method, other methods can also be used with the resampler:



In [16]:
std = aapl.resample('W').std()
max = aapl.resample('W').max()
min = aapl.resample('W').min()


print(std)
print(max)
print(min)

Date
2017-01-08    0.203367
2017-01-15    0.072529
2017-01-22    0.025553
2017-01-29    0.245286
2017-02-05    0.943260
2017-02-12    0.252002
2017-02-19    0.231397
2017-02-26    0.059347
2017-03-05    0.340093
2017-03-12    0.076289
2017-03-19    0.177832
2017-03-26    0.157263
2017-04-02    0.319806
2017-04-09    0.128690
2017-04-16    0.212481
2017-04-23    0.174677
2017-04-30    0.090026
2017-05-07    0.235638
2017-05-14    0.352978
2017-05-21    0.536092
2017-05-28    0.060389
2017-06-04    0.281226
2017-06-11    0.620037
2017-06-18    0.382560
2017-06-25    0.128889
2017-07-02    0.264111
2017-07-09    0.158886
2017-07-16    0.404285
2017-07-23    0.124989
2017-07-30    0.384442
2017-08-06    0.924388
2017-08-13    0.467470
2017-08-20    0.435248
2017-08-27    0.275765
2017-09-03    0.251723
2017-09-10    0.381641
2017-09-17    0.294141
2017-09-24    0.735195
2017-10-01    0.354843
2017-10-08    0.204852
2017-10-15    0.118735
2017-10-22    0.517000
2017-10-29    0.680457
2017-1

Often we want to calculate monthly returns of a stock, based on prices on the last day of each month. To fetch those prices, we use the series.resample.agg() method:

In [17]:
last_day = aapl.resample('M').agg(lambda x: x[-1])
print(last_day)

Date
2017-01-31    28.597813
2017-02-28    32.423561
2017-03-31    34.002262
2017-04-30    33.999897
2017-05-31    36.305340
2017-06-30    34.228165
2017-07-31    35.347557
2017-08-31    39.129723
2017-09-30    36.772392
2017-10-31    40.332249
2017-11-30    41.150108
2017-12-31    40.522736
Freq: M, Name: Close, dtype: float64


Or directly calculate the monthly rates of return using the data for the first day and the last day:

In [19]:
monthly_return = aapl.resample('M').agg(lambda x: x[-1]/x[0] - 1)
print(monthly_return)

Date
2017-01-31    0.044770
2017-02-28    0.068613
2017-03-31    0.027685
2017-04-30   -0.000348
2017-05-31    0.046463
2017-06-30   -0.059799
2017-07-31    0.036446
2017-08-31    0.097261
2017-09-30   -0.060530
2017-10-31    0.099018
2017-11-30    0.033422
2017-12-31   -0.010640
Freq: M, Name: Close, dtype: float64


Series object also provides us some convenient methods to do some quick calculation.

In [20]:
print(monthly_return.mean())
print(monthly_return.std())
print(monthly_return.max())

0.026863389755777945
0.05225845363865124
0.09901816736008073


Another two methods frequently used on Series are .diff() and .pct_change(). The former calculates the difference between consecutive elements, and the latter calculates the percentage change.

In [21]:
print(last_day.diff())
print(last_day.pct_change())

Date
2017-01-31         NaN
2017-02-28    3.825748
2017-03-31    1.578701
2017-04-30   -0.002365
2017-05-31    2.305443
2017-06-30   -2.077175
2017-07-31    1.119392
2017-08-31    3.782166
2017-09-30   -2.357330
2017-10-31    3.559856
2017-11-30    0.817860
2017-12-31   -0.627373
Freq: M, Name: Close, dtype: float64
Date
2017-01-31         NaN
2017-02-28    0.133778
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64


Notice that we induced a NaN value while calculating percentage changes i.e. returns.

When dealing with NaN values, we usually either removing the data point or fill it with a specific value. Here we fill it with 0:

In [22]:
daily_return = last_day.pct_change()
print(daily_return.fillna(0))

Date
2017-01-31    0.000000
2017-02-28    0.133778
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64


Alternatively, we can fill a NaN with the next fitted value. This is called 'backward fill', or 'bfill' in short:

In [23]:
daily_return = last_day.pct_change()
print(daily_return.fillna(method = 'bfill'))

Date
2017-01-31    0.133778
2017-02-28    0.133778
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64


As expected, since there is a 'backward fill' method, there must be a 'forward fill' method, or 'ffill' in short. However we can't use it here because the NaN is the first value.

We can also simply remove NaN values by **_.dropna()_**

In [24]:
daily_return = last_day.pct_change()
daily_return.dropna()

Date
2017-02-28    0.133778
2017-03-31    0.048690
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096808
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64

# DataFrame
The **DataFrame** is the most commonly used data structure in Pandas. It is essentially a table, just like an Excel spreadsheet.

More precisely, a DataFrame is a collection of Series objects, each of which may contain different data types. A DataFrame can be created from various data types: dictionary, 2-D numpy.ndarray, a Series or another DataFrame.

## Create DataFrames
The most common method of creating a DataFrame is passing a dictionary:

In [27]:

import pandas as pd

dict = {'AAPL': [143.5, 144.09, 142.73, 144.18, 143.77],'GOOG':[898.7, 911.71, 906.69, 918.59, 926.99],
        'IBM':[155.58, 153.67, 152.36, 152.94, 153.49]}
data_index = pd.date_range('2017-07-03',periods = 5, freq = 'D')
df = pd.DataFrame(dict, index = data_index)
print(df)

              AAPL    GOOG     IBM
2017-07-03  143.50  898.70  155.58
2017-07-04  144.09  911.71  153.67
2017-07-05  142.73  906.69  152.36
2017-07-06  144.18  918.59  152.94
2017-07-07  143.77  926.99  153.49


## Manipulating DataFrames
We can fetch values in a DataFrame by columns and index. Each column in a DataFrame is essentially a Pandas Series. We can fetch a column by square brackets: **df['column_name']**

If a column name contains no spaces, then we can also use df.column_name to fetch a column:

In [29]:
df = aapl_table
print(df.Close.tail(5))
print(df['Volume'].tail(5))

Date
2017-12-22    41.906773
2017-12-26    40.843597
2017-12-27    40.850788
2017-12-28    40.965721
2017-12-29    40.522736
Name: Close, dtype: float64
Date
2017-12-22     65397600
2017-12-26    132742000
2017-12-27     85992800
2017-12-28     65920800
2017-12-29    103999600
Name: Volume, dtype: int64


All the methods we applied to a Series index such as iloc[], loc[] and resampling methods, can also be applied to a DataFrame:

In [30]:
aapl_2016 = df['2016']
aapl_month = aapl_2016.resample('M').agg(lambda x: x[-1])
print(aapl_month)

                 Open       High        Low  ...     Volume  Dividends  Stock Splits
Date                                         ...                                    
2016-01-31  21.852380  22.440243  21.750945  ...  257666000        0.0             0
2016-02-29  22.450759  22.768306  22.402084  ...  140865200        0.0             0
2016-03-31  25.431520  25.473242  25.236819  ...  103553600        0.0             0
2016-04-30  21.785533  21.954737  21.442491  ...  274126000        0.0             0
2016-05-31  23.226407  23.412965  23.044514  ...  169228800        0.0             0
2016-06-30  22.023112  22.333262  21.990464  ...  143345600        0.0             0
2016-07-31  24.296782  24.380733  24.177851  ...  110934800        0.0             0
2016-08-31  24.773059  24.986417  24.768369  ...  118649600        0.0             0
2016-09-30  26.367388  26.580747  26.212645  ...  145516400        0.0             0
2016-10-31  26.646392  26.782380  26.540884  ...  105677600      

We may select certain columns of a DataFrame using their names:

In [33]:
aapl_bar = aapl_month[['Open', 'High', 'Low', 'Close']]
print(aapl_bar)

                 Open       High        Low      Close
Date                                                  
2016-01-31  21.852380  22.440243  21.750945  22.440243
2016-02-29  22.450759  22.768306  22.402084  22.411356
2016-03-31  25.431520  25.473242  25.236819  25.262316
2016-04-30  21.785533  21.954737  21.442491  21.727587
2016-05-31  23.226407  23.412965  23.044514  23.287039
2016-06-30  22.023112  22.333262  21.990464  22.293619
2016-07-31  24.296782  24.380733  24.177851  24.301445
2016-08-31  24.773059  24.986417  24.768369  24.876221
2016-09-30  26.367388  26.580747  26.212645  26.505720
2016-10-31  26.646392  26.782380  26.540884  26.620602
2016-11-30  26.300090  26.441488  25.986656  26.045572
2016-12-31  27.490193  27.619807  27.202683  27.294592


We can even specify both rows and columns using loc[]. The row indices and column names are separated by a comma:

In [35]:
print(aapl_month.loc['2016-03':'2016-06',['Open', 'High', 'Low', 'Close']])

                 Open       High        Low      Close
Date                                                  
2016-03-31  25.431520  25.473242  25.236819  25.262316
2016-04-30  21.785533  21.954737  21.442491  21.727587
2016-05-31  23.226407  23.412965  23.044514  23.287039
2016-06-30  22.023112  22.333262  21.990464  22.293619


The subset methods in DataFrame is quite useful. By writing logical statements in square brackets, we can make customized subsets:

In [37]:

import numpy as np

above = aapl_bar[aapl_bar.Close > np.mean(aapl_bar.Close)]
print(above)

                 Open       High        Low      Close
Date                                                  
2016-03-31  25.431520  25.473242  25.236819  25.262316
2016-08-31  24.773059  24.986417  24.768369  24.876221
2016-09-30  26.367388  26.580747  26.212645  26.505720
2016-10-31  26.646392  26.782380  26.540884  26.620602
2016-11-30  26.300090  26.441488  25.986656  26.045572
2016-12-31  27.490193  27.619807  27.202683  27.294592


## Data Validation
As mentioned, all methods that apply to a Series can also be applied to a DataFrame. Here we add a new column to an existing DataFrame:

In [39]:
aapl_bar['rate_return'] = aapl_bar.Close.pct_change()
print(aapl_bar)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  21.852380  22.440243  21.750945  22.440243          NaN
2016-02-29  22.450759  22.768306  22.402084  22.411356    -0.001287
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127210
2016-04-30  21.785533  21.954737  21.442491  21.727587    -0.139921
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773
2016-06-30  22.023112  22.333262  21.990464  22.293619    -0.042660
2016-07-31  24.296782  24.380733  24.177851  24.301445     0.090063
2016-08-31  24.773059  24.986417  24.768369  24.876221     0.023652
2016-09-30  26.367388  26.580747  26.212645  26.505720     0.065504
2016-10-31  26.646392  26.782380  26.540884  26.620602     0.004334
2016-11-30  26.300090  26.441488  25.986656  26.045572    -0.021601
2016-12-31  27.490193  27.619807  27.202683  27.294592     0.047955


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Here the calculation introduced a NaN value. If the DataFrame is large, we would not be able to observe it. **isnull()** provides a convenient way to check abnormal values.

In [40]:
missing = aapl_bar.isnull()
print(missing)
print('---------------------------------------------')
print(missing.describe())

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True
2016-02-29  False  False  False  False        False
2016-03-31  False  False  False  False        False
2016-04-30  False  False  False  False        False
2016-05-31  False  False  False  False        False
2016-06-30  False  False  False  False        False
2016-07-31  False  False  False  False        False
2016-08-31  False  False  False  False        False
2016-09-30  False  False  False  False        False
2016-10-31  False  False  False  False        False
2016-11-30  False  False  False  False        False
2016-12-31  False  False  False  False        False
---------------------------------------------
         Open   High    Low  Close rate_return
count      12     12     12     12          12
unique      1      1      1      1           2
top     False  False  False  False       False
freq       12     12     12     12    

The row labelled "unique" indicates the number of unique values in each column. Since the "rate_return" column has 2 unique values, it has at least one missing value.

We can deduce the number of missing values by comparing "count" with "freq". There are 12 counts and 11 False values, so there is one True value which corresponds to the missing value.

We can also find the rows with missing values easily:

In [41]:
print(missing[missing.rate_return == True])

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True


Usually when dealing with missing data, we either delete the whole row or fill it with some value. As we introduced in the Series chapter, the same method **dropna()** and **fillna()** can be applied to a DataFrame.

In [42]:
drop = aapl_bar.dropna()
print(drop)
print('\n--------------------------------------------------\n')
fill = aapl_bar.fillna(0)
print(fill)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-02-29  22.450759  22.768306  22.402084  22.411356    -0.001287
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127210
2016-04-30  21.785533  21.954737  21.442491  21.727587    -0.139921
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773
2016-06-30  22.023112  22.333262  21.990464  22.293619    -0.042660
2016-07-31  24.296782  24.380733  24.177851  24.301445     0.090063
2016-08-31  24.773059  24.986417  24.768369  24.876221     0.023652
2016-09-30  26.367388  26.580747  26.212645  26.505720     0.065504
2016-10-31  26.646392  26.782380  26.540884  26.620602     0.004334
2016-11-30  26.300090  26.441488  25.986656  26.045572    -0.021601
2016-12-31  27.490193  27.619807  27.202683  27.294592     0.047955

--------------------------------------------------

                 Open       High        Low      Close  rate_re

## DataFrame Concat
We have seen how to extract a Series from a dataFrame. Now we need to consider how to merge a Series or a DataFrame into another one.

In Pandas, the function **concat()** allows us to merge multiple Series into a DataFrame:

In [43]:
s1 = pd.Series([143.5, 144.09, 142.73, 144.18, 143.77], name = 'AAPL')
s2 = pd.Series([898.7, 911.71, 906.69, 918.59, 926.99], name = 'GOOG')
data_frame = pd.concat([s1,s2], axis = 1)
print(data_frame)

     AAPL    GOOG
0  143.50  898.70
1  144.09  911.71
2  142.73  906.69
3  144.18  918.59
4  143.77  926.99


The "axis = 1" parameter will join two DataFrames by columns:

In [44]:
log_price = np.log(aapl_bar.Close)
log_price.name = 'log_price'
print(log_price)
print('\n---------------------- separate line--------------------\n')
concat = pd.concat([aapl_bar, log_price], axis = 1)
print(concat)

Date
2016-01-31    3.110856
2016-02-29    3.109568
2016-03-31    3.229314
2016-04-30    3.078583
2016-05-31    3.147897
2016-06-30    3.104301
2016-07-31    3.190536
2016-08-31    3.213912
2016-09-30    3.277361
2016-10-31    3.281685
2016-11-30    3.259848
2016-12-31    3.306689
Freq: M, Name: log_price, dtype: float64

---------------------- separate line--------------------

                 Open       High        Low      Close  rate_return  log_price
Date                                                                          
2016-01-31  21.852380  22.440243  21.750945  22.440243          NaN   3.110856
2016-02-29  22.450759  22.768306  22.402084  22.411356    -0.001287   3.109568
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127210   3.229314
2016-04-30  21.785533  21.954737  21.442491  21.727587    -0.139921   3.078583
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773   3.147897
2016-06-30  22.023112  22.333262  21.990464  22.293619    -0.04266

We can also join two DataFrames by rows. Consider these two DataFrames:

In [46]:

df_volume = aapl_table.loc['2016-10':'2017-04',['Volume', 'Stock Splits']].resample('M').agg(lambda x: x[-1])
print(df_volume)
print('\n---------------------- separate line--------------------\n')
df_2017 = aapl_table.loc['2016-10':'2017-04',['Open', 'High', 'Low', 'Close']].resample('M').agg(lambda x: x[-1])
print(df_2017)

               Volume  Stock Splits
Date                               
2016-10-31  105677600             0
2016-11-30  144649200             0
2016-12-31  122345200             0
2017-01-31  196804000             0
2017-02-28   93931600             0
2017-03-31   78646800             0
2017-04-30   83441600             0

---------------------- separate line--------------------

                 Open       High        Low      Close
Date                                                  
2016-10-31  26.646392  26.782380  26.540884  26.620602
2016-11-30  26.300090  26.441488  25.986656  26.045572
2016-12-31  27.490193  27.619807  27.202683  27.294592
2017-01-31  28.550681  28.607239  28.425779  28.597813
2017-02-28  32.444862  32.530069  32.354920  32.423561
2017-03-31  34.016463  34.146641  33.848414  34.002262
2017-04-30  34.104039  34.153745  33.909959  33.999897


Now we merge the DataFrames with our DataFrame 'aapl_bar'

In [47]:
concat = pd.concat([aapl_bar, df_volume], axis = 1)
print(concat)

                 Open       High  ...       Volume  Stock Splits
Date                              ...                           
2016-01-31  21.852380  22.440243  ...          NaN           NaN
2016-02-29  22.450759  22.768306  ...          NaN           NaN
2016-03-31  25.431520  25.473242  ...          NaN           NaN
2016-04-30  21.785533  21.954737  ...          NaN           NaN
2016-05-31  23.226407  23.412965  ...          NaN           NaN
2016-06-30  22.023112  22.333262  ...          NaN           NaN
2016-07-31  24.296782  24.380733  ...          NaN           NaN
2016-08-31  24.773059  24.986417  ...          NaN           NaN
2016-09-30  26.367388  26.580747  ...          NaN           NaN
2016-10-31  26.646392  26.782380  ...  105677600.0           0.0
2016-11-30  26.300090  26.441488  ...  144649200.0           0.0
2016-12-31  27.490193  27.619807  ...  122345200.0           0.0
2017-01-31        NaN        NaN  ...  196804000.0           0.0
2017-02-28        NaN    

By default the DataFrame are joined with all of the data. This default options results in zero information loss. We can also merge them by intersection, this is called 'inner join

In [48]:
concat = pd.concat([aapl_bar,df_volume],axis = 1, join = 'inner')
print(concat)

                 Open       High  ...     Volume  Stock Splits
Date                              ...                         
2016-10-31  26.646392  26.782380  ...  105677600             0
2016-11-30  26.300090  26.441488  ...  144649200             0
2016-12-31  27.490193  27.619807  ...  122345200             0

[3 rows x 7 columns]


Only the intersection part was left if use 'inner join' method. Now let's try to append a DataFrame to another one:

In [49]:
append = aapl_bar.append(df_2017)
print(append)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  21.852380  22.440243  21.750945  22.440243          NaN
2016-02-29  22.450759  22.768306  22.402084  22.411356    -0.001287
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127210
2016-04-30  21.785533  21.954737  21.442491  21.727587    -0.139921
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773
2016-06-30  22.023112  22.333262  21.990464  22.293619    -0.042660
2016-07-31  24.296782  24.380733  24.177851  24.301445     0.090063
2016-08-31  24.773059  24.986417  24.768369  24.876221     0.023652
2016-09-30  26.367388  26.580747  26.212645  26.505720     0.065504
2016-10-31  26.646392  26.782380  26.540884  26.620602     0.004334
2016-11-30  26.300090  26.441488  25.986656  26.045572    -0.021601
2016-12-31  27.490193  27.619807  27.202683  27.294592     0.047955
2016-10-31  26.646392  26.782380  26.540884  26.

'Append' is essentially to concat two DataFrames by axis = 0, thus here is an alternative way to append:

In [50]:
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print(concat)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  21.852380  22.440243  21.750945  22.440243          NaN
2016-02-29  22.450759  22.768306  22.402084  22.411356    -0.001287
2016-03-31  25.431520  25.473242  25.236819  25.262316     0.127210
2016-04-30  21.785533  21.954737  21.442491  21.727587    -0.139921
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773
2016-06-30  22.023112  22.333262  21.990464  22.293619    -0.042660
2016-07-31  24.296782  24.380733  24.177851  24.301445     0.090063
2016-08-31  24.773059  24.986417  24.768369  24.876221     0.023652
2016-09-30  26.367388  26.580747  26.212645  26.505720     0.065504
2016-10-31  26.646392  26.782380  26.540884  26.620602     0.004334
2016-11-30  26.300090  26.441488  25.986656  26.045572    -0.021601
2016-12-31  27.490193  27.619807  27.202683  27.294592     0.047955
2016-10-31  26.646392  26.782380  26.540884  26.

Please note that if the two DataFrame have some columns with the same column names, these columns are considered to be the same and will be merged. It's very important to have the right column names. If we change a column names here:

In [51]:
df_2017.columns = ['change', 'high', 'low', 'close']
concat = pd.concat([aapl_bar, df_2017], axis = 0,sort=True)
print(concat)

                Close       High        Low  ...       high        low  rate_return
Date                                         ...                                   
2016-01-31  22.440243  22.440243  21.750945  ...        NaN        NaN          NaN
2016-02-29  22.411356  22.768306  22.402084  ...        NaN        NaN    -0.001287
2016-03-31  25.262316  25.473242  25.236819  ...        NaN        NaN     0.127210
2016-04-30  21.727587  21.954737  21.442491  ...        NaN        NaN    -0.139921
2016-05-31  23.287039  23.412965  23.044514  ...        NaN        NaN     0.071773
2016-06-30  22.293619  22.333262  21.990464  ...        NaN        NaN    -0.042660
2016-07-31  24.301445  24.380733  24.177851  ...        NaN        NaN     0.090063
2016-08-31  24.876221  24.986417  24.768369  ...        NaN        NaN     0.023652
2016-09-30  26.505720  26.580747  26.212645  ...        NaN        NaN     0.065504
2016-10-31  26.620602  26.782380  26.540884  ...        NaN        NaN     0

Since the column name of 'Open' has been changed, the new DataFrame has an new column named 'Change'.

# Summary

Hereby we introduced the most import part of python: resampling and DataFrame manipulation. We only introduced the most commonly used method in Financial data analysis. There are also many methods used in data mining, which are also beneficial. You can always check the [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) official documentations for help.

<div align="center">
<img style="display: block; margin: auto;" alt="photo" src="https://cdn.quantconnect.com/web/i/icon.png">

Quantconnect

Introduction to Financial Python
</div>

# 05 Pandas-Resampling and DataFrame

# Introduction
In the last chapter we had a glimpse of Pandas. In this chapter we will learn about resampling methods and the DataFrame object, which is a powerful tool for financial data analysis.

# Fetching Data
Here we use the QuantBook API to retrieve data.



In [139]:
!pip install yfinance


     |████████████████████████████████| 6.3 MB 13.9 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=c62cc88676049352fb06791aba850ce32bc01e0c40cdbf53863697794bfd747e
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [140]:
import yfinance as yf

aapl = yf.Ticker("AAPL")

# get stock info
print(aapl.info)

# get historical market data
aapl_table = aapl.history(start="2016-01-01",  end="2017-12-31")
aapl_table


{'zip': '95014', 'sector': 'Technology', 'fullTimeEmployees': 147000, 'longBusinessSummary': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, iPod touch, and other Apple-branded and third-party accessories. It also provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store, that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand radi

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2016-01-04,23.655164,24.291441,23.514538,24.286829,270597600,0.0,0
2016-01-05,24.379045,24.402098,23.609059,23.678219,223164000,0.0,0
2016-01-06,23.182573,23.599842,23.023505,23.214848,273829600,0.0,0
2016-01-07,22.749166,23.083441,22.230463,22.235073,324377600,0.0,0
2016-01-08,22.719191,22.848290,22.306534,22.352640,283192000,0.0,0
...,...,...,...,...,...,...,...
2017-12-22,41.827752,42.004949,41.784653,41.906773,65397600,0.0,0
2017-12-26,40.898675,41.059108,40.630484,40.843601,132742000,0.0,0
2017-12-27,40.731065,40.893892,40.637678,40.850792,85992800,0.0,0


In [142]:
aapl = aapl_table['Close']['2017']


In [143]:
print(aapl)

Date
2017-01-03    27.372362
2017-01-04    27.341726
2017-01-05    27.480770
2017-01-06    27.787130
2017-01-09    28.041653
                ...    
2017-12-22    41.906773
2017-12-26    40.843601
2017-12-27    40.850792
2017-12-28    40.965717
2017-12-29    40.522732
Name: Close, Length: 251, dtype: float64


Recall that we can fetch a specific data point using series['yyyy-mm-dd']. We can also fetch the data in a specific month using series['yyyy-mm'].

In [144]:
print(aapl['2017-3'])

Date
2017-03-01    33.086281
2017-03-02    32.889839
2017-03-03    33.083923
2017-03-06    32.979771
2017-03-07    33.022385
2017-03-08    32.899303
2017-03-09    32.823563
2017-03-10    32.932442
2017-03-13    32.946636
2017-03-14    32.896935
2017-03-15    33.244869
2017-03-16    33.299313
2017-03-17    33.133621
2017-03-20    33.481548
2017-03-21    33.098114
2017-03-22    33.472080
2017-03-23    33.353741
2017-03-24    33.287472
2017-03-27    33.344280
2017-03-28    34.035400
2017-03-29    34.111141
2017-03-30    34.066166
2017-03-31    34.002254
Name: Close, dtype: float64


In [145]:
aapl['2017-2':'2017-4']

Date
2017-02-01    30.341724
2017-02-02    30.289883
2017-02-03    30.419493
2017-02-06    30.704645
2017-02-07    30.996874
                ...    
2017-04-24    33.997524
2017-04-25    34.208172
2017-04-26    34.006989
2017-04-27    34.033024
2017-04-28    33.999889
Name: Close, Length: 61, dtype: float64

.head(N) and .tail(N) are methods for quickly accessing the first or last N elements.

In [146]:
print(aapl.head(5))
print(aapl.tail(10))

Date
2017-01-03    27.372362
2017-01-04    27.341726
2017-01-05    27.480770
2017-01-06    27.787130
2017-01-09    28.041653
Name: Close, dtype: float64
Date
2017-12-15    41.657742
2017-12-18    42.244400
2017-12-19    41.794235
2017-12-20    41.748734
2017-12-21    41.906773
2017-12-22    41.906773
2017-12-26    40.843601
2017-12-27    40.850792
2017-12-28    40.965717
2017-12-29    40.522732
Name: Close, dtype: float64


# Resampling
**_series.resample(freq)_** is a class called "DatetimeIndexResampler" which groups data in a Series object into regular time intervals. The argument "freq" determines the length of each interval.

**_series.resample.mean()_** is a complete statement that groups data into intervals, and then compute the mean of each interval. For example, if we want to aggregate the daily data into monthly data by mean:

In [147]:
by_month = aapl.resample('M').mean()
print(by_month)

Date
2017-01-31    28.178333
2017-02-28    31.606276
2017-03-31    33.282221
2017-04-30    33.819263
2017-05-31    36.125684
2017-06-30    35.133984
2017-07-31    35.245242
2017-08-31    37.897227
2017-09-30    37.604739
2017-10-31    37.654550
2017-11-30    41.233915
2017-12-31    41.160039
Freq: M, Name: Close, dtype: float64


We can also aggregate the data by week:

In [148]:
by_week = aapl.resample('W').mean()
print(by_week.head())

Date
2017-01-08    27.495497
2017-01-15    28.097735
2017-01-22    28.266119
2017-01-29    28.553980
2017-02-05    29.662542
Freq: W-SUN, Name: Close, dtype: float64


We can also aggregate the data by month with max:

In [149]:
aapl.resample('M').max()

Date
2017-01-31    28.739210
2017-02-28    32.451969
2017-03-31    34.111141
2017-04-30    34.264984
2017-05-31    37.099129
2017-06-30    36.944653
2017-07-31    36.471703
2017-08-31    39.129726
2017-09-30    39.141655
2017-10-31    40.332241
2017-11-30    42.050133
2017-12-31    42.244400
Freq: M, Name: Close, dtype: float64

We can choose almost any frequency by using the format 'nf', where 'n' is an integer and 'f' is M for month, W for week and D for day.

In [150]:
three_day = aapl.resample('3D').mean()
two_week = aapl.resample('2W').mean()
two_month = aapl.resample('2M').mean()


print(three_day)
print(two_week)
print(two_month )

Date
2017-01-03    27.398286
2017-01-06    27.787130
2017-01-09    28.110776
2017-01-12    28.078174
2017-01-15    28.279671
                ...    
2017-12-17    42.019318
2017-12-20    41.854093
2017-12-23          NaN
2017-12-26    40.886703
2017-12-29    40.522732
Freq: 3D, Name: Close, Length: 121, dtype: float64
Date
2017-01-08    27.495497
2017-01-22    28.172572
2017-02-05    29.108261
2017-02-19    31.516816
2017-03-05    32.595558
2017-03-19    33.017884
2017-04-02    33.625220
2017-04-16    33.849728
2017-04-30    33.791845
2017-05-14    35.697633
2017-05-28    36.496176
2017-06-11    36.515010
2017-06-25    34.529044
2017-07-09    34.265135
2017-07-23    35.278872
2017-08-06    36.272775
2017-08-20    37.904118
2017-09-03    38.458792
2017-09-17    38.282708
2017-10-01    36.840875
2017-10-15    37.071595
2017-10-29    37.757320
2017-11-12    41.032237
2017-11-26    41.204376
2017-12-10    40.867068
2017-12-24    41.620866
2018-01-07    40.795711
Freq: 2W-SUN, Name: Close, 

Besides the mean() method, other methods can also be used with the resampler:



In [151]:
std = aapl.resample('W').std()
max = aapl.resample('W').max()
min = aapl.resample('W').min()


print(std)
print(max)
print(min)

Date
2017-01-08    0.203367
2017-01-15    0.072528
2017-01-22    0.025554
2017-01-29    0.245284
2017-02-05    0.943260
2017-02-12    0.252001
2017-02-19    0.231394
2017-02-26    0.059347
2017-03-05    0.340089
2017-03-12    0.076286
2017-03-19    0.177840
2017-03-26    0.157264
2017-04-02    0.319806
2017-04-09    0.128692
2017-04-16    0.212473
2017-04-23    0.174675
2017-04-30    0.090024
2017-05-07    0.235639
2017-05-14    0.352978
2017-05-21    0.536094
2017-05-28    0.060388
2017-06-04    0.281225
2017-06-11    0.620035
2017-06-18    0.382563
2017-06-25    0.128896
2017-07-02    0.264107
2017-07-09    0.158887
2017-07-16    0.404286
2017-07-23    0.124991
2017-07-30    0.384442
2017-08-06    0.924387
2017-08-13    0.467469
2017-08-20    0.435247
2017-08-27    0.275765
2017-09-03    0.251727
2017-09-10    0.381638
2017-09-17    0.294139
2017-09-24    0.735188
2017-10-01    0.354846
2017-10-08    0.204854
2017-10-15    0.118728
2017-10-22    0.516996
2017-10-29    0.680459
2017-1

In [152]:
last_day = aapl.resample('M').agg(lambda x: x[-1])
print(last_day)

Date
2017-01-31    28.597815
2017-02-28    32.423573
2017-03-31    34.002254
2017-04-30    33.999889
2017-05-31    36.305336
2017-06-30    34.228172
2017-07-31    35.347565
2017-08-31    39.129726
2017-09-30    36.772400
2017-10-31    40.332241
2017-11-30    41.150105
2017-12-31    40.522732
Freq: M, Name: Close, dtype: float64


In [153]:
monthly_return = aapl.resample('M').agg(lambda x: x[-1]/x[0] - 1)
print(monthly_return)

Date
2017-01-31    0.044770
2017-02-28    0.068613
2017-03-31    0.027684
2017-04-30   -0.000348
2017-05-31    0.046463
2017-06-30   -0.059799
2017-07-31    0.036446
2017-08-31    0.097261
2017-09-30   -0.060530
2017-10-31    0.099018
2017-11-30    0.033422
2017-12-31   -0.010640
Freq: M, Name: Close, dtype: float64


In [154]:
print(monthly_return.mean())
print(monthly_return.std())
print(monthly_return.max())

0.026863401635711603
0.05225848509209957
0.09901818794539397


In [155]:
print(last_day.diff())
print(last_day.pct_change())

Date
2017-01-31         NaN
2017-02-28    3.825758
2017-03-31    1.578682
2017-04-30   -0.002365
2017-05-31    2.305447
2017-06-30   -2.077164
2017-07-31    1.119392
2017-08-31    3.782162
2017-09-30   -2.357327
2017-10-31    3.559841
2017-11-30    0.817863
2017-12-31   -0.627373
Freq: M, Name: Close, dtype: float64
Date
2017-01-31         NaN
2017-02-28    0.133778
2017-03-31    0.048689
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096807
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64


Notice that we induced a NaN value while calculating percentage changes i.e. returns.

When dealing with NaN values, we usually either removing the data point or fill it with a specific value. Here we fill it with 0:

In [156]:
daily_return = last_day.pct_change()
print(daily_return.fillna(0))

Date
2017-01-31    0.000000
2017-02-28    0.133778
2017-03-31    0.048689
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096807
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64


In [157]:
daily_return = last_day.pct_change()
print(daily_return.fillna(method = 'bfill'))

Date
2017-01-31    0.133778
2017-02-28    0.133778
2017-03-31    0.048689
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096807
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64


In [158]:
daily_return = last_day.pct_change()
daily_return.dropna()

Date
2017-02-28    0.133778
2017-03-31    0.048689
2017-04-30   -0.000070
2017-05-31    0.067807
2017-06-30   -0.057214
2017-07-31    0.032704
2017-08-31    0.106999
2017-09-30   -0.060244
2017-10-31    0.096807
2017-11-30    0.020278
2017-12-31   -0.015246
Freq: M, Name: Close, dtype: float64

# DataFrame
The **DataFrame** is the most commonly used data structure in Pandas. It is essentially a table, just like an Excel spreadsheet.

More precisely, a DataFrame is a collection of Series objects, each of which may contain different data types. A DataFrame can be created from various data types: dictionary, 2-D numpy.ndarray, a Series or another DataFrame.

## Create DataFrames
The most common method of creating a DataFrame is passing a dictionary:

In [159]:
import pandas as pd

dict = {'AAPL': [143.5, 144.09, 142.73, 144.18, 143.77],'GOOG':[898.7, 911.71, 906.69, 918.59, 926.99],
        'IBM':[155.58, 153.67, 152.36, 152.94, 153.49]}
data_index = pd.date_range('2017-07-03',periods = 5, freq = 'D')
df = pd.DataFrame(dict, index = data_index)
print(df)

              AAPL    GOOG     IBM
2017-07-03  143.50  898.70  155.58
2017-07-04  144.09  911.71  153.67
2017-07-05  142.73  906.69  152.36
2017-07-06  144.18  918.59  152.94
2017-07-07  143.77  926.99  153.49


## Manipulating DataFrames
We can fetch values in a DataFrame by columns and index. Each column in a DataFrame is essentially a Pandas Series. We can fetch a column by square brackets: **df['column_name']**

If a column name contains no spaces, then we can also use df.column_name to fetch a column:

In [160]:
df = aapl_table
print(df.Close.tail(5))
print(df['Volume'].tail(5))

Date
2017-12-22    41.906773
2017-12-26    40.843601
2017-12-27    40.850792
2017-12-28    40.965717
2017-12-29    40.522732
Name: Close, dtype: float64
Date
2017-12-22     65397600
2017-12-26    132742000
2017-12-27     85992800
2017-12-28     65920800
2017-12-29    103999600
Name: Volume, dtype: int64


All the methods we applied to a Series index such as iloc[], loc[] and resampling methods, can also be applied to a DataFrame:

In [161]:
aapl_2016 = df['2016']
aapl_month = aapl_2016.resample('M').agg(lambda x: x[-1])
print(aapl_month)

                 Open       High        Low  ...     Volume  Dividends  Stock Splits
Date                                         ...                                    
2016-01-31  21.852379  22.440241  21.750943  ...  257666000        0.0             0
2016-02-29  22.450755  22.768302  22.402081  ...  140865200        0.0             0
2016-03-31  25.431526  25.473247  25.236825  ...  103553600        0.0             0
2016-04-30  21.785535  21.954739  21.442493  ...  274126000        0.0             0
2016-05-31  23.226407  23.412965  23.044514  ...  169228800        0.0             0
2016-06-30  22.023106  22.333256  21.990459  ...  143345600        0.0             0
2016-07-31  24.296784  24.380735  24.177853  ...  110934800        0.0             0
2016-08-31  24.773056  24.986413  24.768365  ...  118649600        0.0             0
2016-09-30  26.367390  26.580749  26.212647  ...  145516400        0.0             0
2016-10-31  26.646398  26.782385  26.540890  ...  105677600      

In [162]:
aapl_bar = aapl_month[['Open', 'High', 'Low', 'Close']]
print(aapl_bar)

                 Open       High        Low      Close
Date                                                  
2016-01-31  21.852379  22.440241  21.750943  22.440241
2016-02-29  22.450755  22.768302  22.402081  22.411352
2016-03-31  25.431526  25.473247  25.236825  25.262321
2016-04-30  21.785535  21.954739  21.442493  21.727589
2016-05-31  23.226407  23.412965  23.044514  23.287039
2016-06-30  22.023106  22.333256  21.990459  22.293613
2016-07-31  24.296784  24.380735  24.177853  24.301447
2016-08-31  24.773056  24.986413  24.768365  24.876217
2016-09-30  26.367390  26.580749  26.212647  26.505722
2016-10-31  26.646398  26.782385  26.540890  26.620607
2016-11-30  26.300092  26.441490  25.986658  26.045574
2016-12-31  27.490197  27.619811  27.202687  27.294596


In [163]:
print(aapl_month.loc['2016-03':'2016-06',['Open', 'High', 'Low', 'Close']])

                 Open       High        Low      Close
Date                                                  
2016-03-31  25.431526  25.473247  25.236825  25.262321
2016-04-30  21.785535  21.954739  21.442493  21.727589
2016-05-31  23.226407  23.412965  23.044514  23.287039
2016-06-30  22.023106  22.333256  21.990459  22.293613


In [164]:
import numpy as np

above = aapl_bar[aapl_bar.Close > np.mean(aapl_bar.Close)]
print(above)

                 Open       High        Low      Close
Date                                                  
2016-03-31  25.431526  25.473247  25.236825  25.262321
2016-08-31  24.773056  24.986413  24.768365  24.876217
2016-09-30  26.367390  26.580749  26.212647  26.505722
2016-10-31  26.646398  26.782385  26.540890  26.620607
2016-11-30  26.300092  26.441490  25.986658  26.045574
2016-12-31  27.490197  27.619811  27.202687  27.294596


## Data Validation
As mentioned, all methods that apply to a Series can also be applied to a DataFrame. Here we add a new column to an existing DataFrame:

In [165]:
aapl_bar['rate_return'] = aapl_bar.Close.pct_change()
print(aapl_bar)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  21.852379  22.440241  21.750943  22.440241          NaN
2016-02-29  22.450755  22.768302  22.402081  22.411352    -0.001287
2016-03-31  25.431526  25.473247  25.236825  25.262321     0.127211
2016-04-30  21.785535  21.954739  21.442493  21.727589    -0.139921
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773
2016-06-30  22.023106  22.333256  21.990459  22.293613    -0.042660
2016-07-31  24.296784  24.380735  24.177853  24.301447     0.090063
2016-08-31  24.773056  24.986413  24.768365  24.876217     0.023652
2016-09-30  26.367390  26.580749  26.212647  26.505722     0.065505
2016-10-31  26.646398  26.782385  26.540890  26.620607     0.004334
2016-11-30  26.300092  26.441490  25.986658  26.045574    -0.021601
2016-12-31  27.490197  27.619811  27.202687  27.294596     0.047955


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Here the calculation introduced a NaN value. If the DataFrame is large, we would not be able to observe it. **isnull()** provides a convenient way to check abnormal values.

In [166]:
missing = aapl_bar.isnull()
print(missing)
print('---------------------------------------------')
print(missing.describe())

             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True
2016-02-29  False  False  False  False        False
2016-03-31  False  False  False  False        False
2016-04-30  False  False  False  False        False
2016-05-31  False  False  False  False        False
2016-06-30  False  False  False  False        False
2016-07-31  False  False  False  False        False
2016-08-31  False  False  False  False        False
2016-09-30  False  False  False  False        False
2016-10-31  False  False  False  False        False
2016-11-30  False  False  False  False        False
2016-12-31  False  False  False  False        False
---------------------------------------------
         Open   High    Low  Close rate_return
count      12     12     12     12          12
unique      1      1      1      1           2
top     False  False  False  False       False
freq       12     12     12     12    

The row labelled "unique" indicates the number of unique values in each column. Since the "rate_return" column has 2 unique values, it has at least one missing value.

We can deduce the number of missing values by comparing "count" with "freq". There are 12 counts and 11 False values, so there is one True value which corresponds to the missing value.

We can also find the rows with missing values easily:

In [167]:
print(missing[missing.rate_return == True])



             Open   High    Low  Close  rate_return
Date                                               
2016-01-31  False  False  False  False         True


Usually when dealing with missing data, we either delete the whole row or fill it with some value. As we introduced in the Series chapter, the same method **dropna()** and **fillna()** can be applied to a DataFrame.

In [168]:
drop = aapl_bar.dropna()
print(drop)
print('\n--------------------------------------------------\n')
fill = aapl_bar.fillna(0)
print(fill)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-02-29  22.450755  22.768302  22.402081  22.411352    -0.001287
2016-03-31  25.431526  25.473247  25.236825  25.262321     0.127211
2016-04-30  21.785535  21.954739  21.442493  21.727589    -0.139921
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773
2016-06-30  22.023106  22.333256  21.990459  22.293613    -0.042660
2016-07-31  24.296784  24.380735  24.177853  24.301447     0.090063
2016-08-31  24.773056  24.986413  24.768365  24.876217     0.023652
2016-09-30  26.367390  26.580749  26.212647  26.505722     0.065505
2016-10-31  26.646398  26.782385  26.540890  26.620607     0.004334
2016-11-30  26.300092  26.441490  25.986658  26.045574    -0.021601
2016-12-31  27.490197  27.619811  27.202687  27.294596     0.047955

--------------------------------------------------

                 Open       High        Low      Close  rate_re

## DataFrame Concat
We have seen how to extract a Series from a dataFrame. Now we need to consider how to merge a Series or a DataFrame into another one.

In Pandas, the function **concat()** allows us to merge multiple Series into a DataFrame:

In [169]:
s1 = pd.Series([143.5, 144.09, 142.73, 144.18, 143.77], name = 'AAPL')
s2 = pd.Series([898.7, 911.71, 906.69, 918.59, 926.99], name = 'GOOG')
data_frame = pd.concat([s1,s2], axis = 1)
print(data_frame)

     AAPL    GOOG
0  143.50  898.70
1  144.09  911.71
2  142.73  906.69
3  144.18  918.59
4  143.77  926.99


The "axis = 1" parameter will join two DataFrames by columns:

In [170]:
log_price = np.log(aapl_bar.Close)
log_price.name = 'log_price'
print(log_price)
print('\n---------------------- separate line--------------------\n')
concat = pd.concat([aapl_bar, log_price], axis = 1)
print(concat)

Date
2016-01-31    3.110856
2016-02-29    3.109568
2016-03-31    3.229314
2016-04-30    3.078583
2016-05-31    3.147897
2016-06-30    3.104300
2016-07-31    3.190536
2016-08-31    3.213912
2016-09-30    3.277361
2016-10-31    3.281686
2016-11-30    3.259848
2016-12-31    3.306689
Freq: M, Name: log_price, dtype: float64

---------------------- separate line--------------------

                 Open       High        Low      Close  rate_return  log_price
Date                                                                          
2016-01-31  21.852379  22.440241  21.750943  22.440241          NaN   3.110856
2016-02-29  22.450755  22.768302  22.402081  22.411352    -0.001287   3.109568
2016-03-31  25.431526  25.473247  25.236825  25.262321     0.127211   3.229314
2016-04-30  21.785535  21.954739  21.442493  21.727589    -0.139921   3.078583
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773   3.147897
2016-06-30  22.023106  22.333256  21.990459  22.293613    -0.04266

In [171]:

df_volume = aapl_table.loc['2016-10':'2017-04',['Volume', 'Stock Splits']].resample('M').agg(lambda x: x[-1])
print(df_volume)
print('\n---------------------- separate line--------------------\n')
df_2017 = aapl_table.loc['2016-10':'2017-04',['Open', 'High', 'Low', 'Close']].resample('M').agg(lambda x: x[-1])
print(df_2017)

               Volume  Stock Splits
Date                               
2016-10-31  105677600             0
2016-11-30  144649200             0
2016-12-31  122345200             0
2017-01-31  196804000             0
2017-02-28   93931600             0
2017-03-31   78646800             0
2017-04-30   83441600             0

---------------------- separate line--------------------

                 Open       High        Low      Close
Date                                                  
2016-10-31  26.646398  26.782385  26.540890  26.620607
2016-11-30  26.300092  26.441490  25.986658  26.045574
2016-12-31  27.490197  27.619811  27.202687  27.294596
2017-01-31  28.550682  28.607241  28.425781  28.597815
2017-02-28  32.444873  32.530080  32.354932  32.423573
2017-03-31  34.016455  34.146633  33.848407  34.002254
2017-04-30  34.104032  34.153737  33.909951  33.999889


In [172]:
concat = pd.concat([aapl_bar, df_volume], axis = 1)
print(concat)

                 Open       High  ...       Volume  Stock Splits
Date                              ...                           
2016-01-31  21.852379  22.440241  ...          NaN           NaN
2016-02-29  22.450755  22.768302  ...          NaN           NaN
2016-03-31  25.431526  25.473247  ...          NaN           NaN
2016-04-30  21.785535  21.954739  ...          NaN           NaN
2016-05-31  23.226407  23.412965  ...          NaN           NaN
2016-06-30  22.023106  22.333256  ...          NaN           NaN
2016-07-31  24.296784  24.380735  ...          NaN           NaN
2016-08-31  24.773056  24.986413  ...          NaN           NaN
2016-09-30  26.367390  26.580749  ...          NaN           NaN
2016-10-31  26.646398  26.782385  ...  105677600.0           0.0
2016-11-30  26.300092  26.441490  ...  144649200.0           0.0
2016-12-31  27.490197  27.619811  ...  122345200.0           0.0
2017-01-31        NaN        NaN  ...  196804000.0           0.0
2017-02-28        NaN    

By default the DataFrame are joined with all of the data. This default options results in zero information loss. We can also merge them by intersection, this is called 'inner join

In [173]:
concat = pd.concat([aapl_bar,df_volume],axis = 1, join = 'inner')
print(concat)

                 Open       High  ...     Volume  Stock Splits
Date                              ...                         
2016-10-31  26.646398  26.782385  ...  105677600             0
2016-11-30  26.300092  26.441490  ...  144649200             0
2016-12-31  27.490197  27.619811  ...  122345200             0

[3 rows x 7 columns]


In [174]:
append = aapl_bar.append(df_2017)
print(append)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  21.852379  22.440241  21.750943  22.440241          NaN
2016-02-29  22.450755  22.768302  22.402081  22.411352    -0.001287
2016-03-31  25.431526  25.473247  25.236825  25.262321     0.127211
2016-04-30  21.785535  21.954739  21.442493  21.727589    -0.139921
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773
2016-06-30  22.023106  22.333256  21.990459  22.293613    -0.042660
2016-07-31  24.296784  24.380735  24.177853  24.301447     0.090063
2016-08-31  24.773056  24.986413  24.768365  24.876217     0.023652
2016-09-30  26.367390  26.580749  26.212647  26.505722     0.065505
2016-10-31  26.646398  26.782385  26.540890  26.620607     0.004334
2016-11-30  26.300092  26.441490  25.986658  26.045574    -0.021601
2016-12-31  27.490197  27.619811  27.202687  27.294596     0.047955
2016-10-31  26.646398  26.782385  26.540890  26.

In [175]:
concat = pd.concat([aapl_bar, df_2017], axis = 0)
print(concat)

                 Open       High        Low      Close  rate_return
Date                                                               
2016-01-31  21.852379  22.440241  21.750943  22.440241          NaN
2016-02-29  22.450755  22.768302  22.402081  22.411352    -0.001287
2016-03-31  25.431526  25.473247  25.236825  25.262321     0.127211
2016-04-30  21.785535  21.954739  21.442493  21.727589    -0.139921
2016-05-31  23.226407  23.412965  23.044514  23.287039     0.071773
2016-06-30  22.023106  22.333256  21.990459  22.293613    -0.042660
2016-07-31  24.296784  24.380735  24.177853  24.301447     0.090063
2016-08-31  24.773056  24.986413  24.768365  24.876217     0.023652
2016-09-30  26.367390  26.580749  26.212647  26.505722     0.065505
2016-10-31  26.646398  26.782385  26.540890  26.620607     0.004334
2016-11-30  26.300092  26.441490  25.986658  26.045574    -0.021601
2016-12-31  27.490197  27.619811  27.202687  27.294596     0.047955
2016-10-31  26.646398  26.782385  26.540890  26.

Please note that if the two DataFrame have some columns with the same column names, these columns are considered to be the same and will be merged. It's very important to have the right column names. If we change a column names here:

In [176]:
df_2017.columns = ['change', 'high', 'low', 'close']
concat = pd.concat([aapl_bar, df_2017], axis = 0,sort=True)
print(concat)

                Close       High        Low  ...       high        low  rate_return
Date                                         ...                                   
2016-01-31  22.440241  22.440241  21.750943  ...        NaN        NaN          NaN
2016-02-29  22.411352  22.768302  22.402081  ...        NaN        NaN    -0.001287
2016-03-31  25.262321  25.473247  25.236825  ...        NaN        NaN     0.127211
2016-04-30  21.727589  21.954739  21.442493  ...        NaN        NaN    -0.139921
2016-05-31  23.287039  23.412965  23.044514  ...        NaN        NaN     0.071773
2016-06-30  22.293613  22.333256  21.990459  ...        NaN        NaN    -0.042660
2016-07-31  24.301447  24.380735  24.177853  ...        NaN        NaN     0.090063
2016-08-31  24.876217  24.986413  24.768365  ...        NaN        NaN     0.023652
2016-09-30  26.505722  26.580749  26.212647  ...        NaN        NaN     0.065505
2016-10-31  26.620607  26.782385  26.540890  ...        NaN        NaN     0

Since the column name of 'Open' has been changed, the new DataFrame has an new column named 'Change'.

# Summary

Hereby we introduced the most import part of python: resampling and DataFrame manipulation. We only introduced the most commonly used method in Financial data analysis. There are also many methods used in data mining, which are also beneficial. You can always check the [Pandas](https://pandas.pydata.org/pandas-docs/stable/index.html) official documentations for help.